In [39]:
#imports
from flair.embeddings import WordEmbeddings
from flair.data import Sentence
import networkx as nx
import torch
from networkx.algorithms import approximation
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm
from numpy import dot
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances
from tqdm.notebook import tqdm
import pickle
from nltk.stem import WordNetLemmatizer
import collections

In [40]:
word_thresh = 0.8
visual_thresh = 0.8
lancaster_thresh = 0.97

# Read child vocabulary 

In [41]:
words = [line.strip() for line in open('../data/vertomul.txt')]
print(len(words))

529


# Recreate the Multiplex Network

In [42]:
def mean_degree_connectivity(graph_name):
    num_edges = 0
    for node in graph_name.nodes():
        num_edges += len(graph_name.edges(node))

    return num_edges/graph_name.number_of_nodes()

In [43]:
def perc_nodes_in_lcc(graph_name):
    largest_cc = max(nx.connected_components(graph_name), key=len)
    lenght = len(largest_cc)
    return lenght/graph_name.number_of_nodes()
   

In [44]:
def mean_shortest_path_lcc(graph_name):
    S = [graph_name.subgraph(c).copy() for c in nx.connected_components(graph_name)]
    comps = [len(max(nx.connected_components(i), key=len)) for i in S]
    index_max = max(range(len(comps)), key=comps.__getitem__)
    return nx.average_shortest_path_length(S[index_max])


## Get the data for the 4 layers

In [45]:
phonological_sim_words = [line.strip() for line in open('../data/phcmul.txt')]
feature_sharing_words = [line.strip() for line in open('../data/mrmul.txt')]
free_association_words = [line.strip() for line in open('../data/famul.txt')]
co_occurrances_words = [line.strip() for line in open('../data/cumul.txt')]

## Clean up the data and store it into lists

In [46]:
phonological_sim_list = []
feature_sharing_list = []
free_association_list = []
co_occurrances_list = []

for pair in phonological_sim_words:
    phonological_sim_list.append(tuple(map(str, pair.split('\t'))))
    
for pair in feature_sharing_words:
    feature_sharing_list.append(tuple(map(str, pair.split('\t'))))
    
for pair in free_association_words:
    free_association_list.append(tuple(map(str, pair.split('\t'))))
    
for pair in co_occurrances_words :
    co_occurrances_list.append(tuple(map(str, pair.split('\t'))))



## Instantiate a graph for multiplex and populate it

In [47]:
multiplex = nx.Graph()
multiplex.add_nodes_from(words)

## We'll add one layer at a time:

## Free associations Layer

In [48]:
free_assoc = nx.Graph()
free_assoc.add_nodes_from(words)
len(free_assoc.nodes)

529

In [49]:
# Add the free association layer to the multiplex
for pair in free_association_list:
    multiplex.add_edge(pair[0], pair[1])
    
# Add the free association layer to its graph
for pair in free_association_list:
    free_assoc.add_edge(pair[0], pair[1])

In [50]:
len(free_assoc.nodes)

529

### 1) Mean degree of connectivity _k_

In [51]:
mean_degree_connectivity(free_assoc)

9.228733459357278

### 2) Mean Clustering Coefficient _CC_

In [52]:
nx.average_clustering(free_assoc)

0.1941295782803327

### 3) Assortativity Coefficient _a_

In [53]:
nx.degree_assortativity_coefficient(free_assoc)

-0.10131692039561056

### 4) Percentage of nodes in the Largest Connected Component _Conn._

In [54]:
perc_nodes_in_lcc(free_assoc)

0.996219281663516

### 5) Mean Shortest Path lenght in the Largest Connected Component _d_

In [55]:
mean_shortest_path_lcc(free_assoc)

3.1650493142185123

### We have added one layer to multiplex

In [56]:
multiplex.number_of_edges()

2441

In [57]:
pickle.dump(free_association_list, open( "free_assoc.pickle", "wb" ) )

In [58]:
len(free_assoc.nodes)

529

## Feature Sharing Norms Layer

In [59]:
feat_norms = nx.Graph()
feat_norms.add_nodes_from(words)

In [60]:
for pair in feature_sharing_list:
        multiplex.add_edge(pair[0], pair[1])
    
for pair in feature_sharing_list:
    feat_norms.add_edge(pair[0], pair[1])

In [61]:
len(feat_norms.nodes)

529

### 1) Mean degree of connectivity _k_

In [62]:
mean_degree_connectivity(feat_norms)

9.032136105860113

### 2) Mean Clustering Coefficient _CC_

In [63]:
nx.average_clustering(feat_norms)

0.1520222342517882

### 3) Assortativity Coefficient _a_

In [64]:
nx.degree_assortativity_coefficient(feat_norms)

-0.010608927346368343

### 4) Percentage of nodes in the Largest Connected Component _Conn._

In [65]:
perc_nodes_in_lcc(feat_norms)

0.24196597353497165

### 5) Mean Shortest Path lenght in the Largest Connected Component _d_

In [66]:
mean_shortest_path_lcc(feat_norms)

1.7570127952755905

### We have added two layers to multiplex

In [67]:
multiplex.number_of_edges()

4686

In [68]:
pickle.dump(feature_sharing_list, open( "feat_norms.pickle", "wb" ) )

In [69]:
len(feat_norms.nodes)

529

## Co-occurrences Layer

In [70]:
co_oc = nx.Graph()
co_oc.add_nodes_from(words)

In [71]:
for pair in co_occurrances_list:
    multiplex.add_edge(pair[0], pair[1])
    
for pair in co_occurrances_list:
    co_oc.add_edge(pair[0], pair[1])

In [72]:
len(co_oc.nodes)

529

### 1) Mean Degree of connectivity _k_

In [73]:
mean_degree_connectivity(co_oc)

8.117202268431003

### 2) Mean Clustering Coefficient _CC_

In [74]:
nx.average_clustering(co_oc)

0.34767890644326055

### 3) Assortativity Coefficient _a_

In [75]:
nx.degree_assortativity_coefficient(co_oc)

-0.43924523453557973

### 4) Percentage of nodes in the Largest Connected Component _Conn._

In [76]:
perc_nodes_in_lcc(co_oc)

0.6238185255198487

### 5) Mean shortest path length of the Largest Connect Component _d_

In [77]:
mean_shortest_path_lcc(co_oc)

2.154517822602929

### We have added three layers to multiplex

In [78]:
multiplex.number_of_edges()

6696

In [79]:
pickle.dump(co_occurrances_list, open( "co_oc.pickle", "wb" ))

In [80]:
len(co_oc.nodes)

529

## Phonological Layer

In [81]:
phon_conn = nx.Graph()
phon_conn.add_nodes_from(words)

In [82]:
for pair in phonological_sim_list:
    multiplex.add_edge(pair[0], pair[1])

for pair in phonological_sim_list:
     phon_conn.add_edge(pair[0], pair[1])

In [83]:
len(phon_conn.nodes)

529

### 1) Mean Degree of Connectivity _a_

In [84]:
mean_degree_connectivity(phon_conn)

1.3194706994328922

### 2) Mean Clustering Coefficient _CC_

In [85]:
nx.average_clustering(phon_conn)

0.1176973624988748

### 3) Assortativity Coefficient _a_

In [86]:
nx.degree_assortativity_coefficient(phon_conn)

0.4824768300873068

### 4) Percentage of nodes in the Largest Connected Component _Conn._

In [87]:
perc_nodes_in_lcc(phon_conn)

0.33081285444234404

### 5) Mean shortest path in the Largest Connected Component _d_

In [88]:
mean_shortest_path_lcc(phon_conn)

7.694844006568145

In [89]:
pickle.dump(phonological_sim_list, open( "phon_conn.pickle", "wb" ))

### We have added all layers to the multiplex

In [90]:
multiplex.number_of_edges()

6998

In [91]:
len(phon_conn.nodes)

529

## The original multiplex layer has been populated

### 1) Mean degree of connectivity _k_

In [92]:
mean_degree_connectivity(multiplex)

26.457466918714555

### 2) Mean Clustering Coefficient _CC_

In [93]:
nx.average_clustering(multiplex)

0.3279300913138615

### 3) Assortativity Coefficient _a_

In [94]:
nx.degree_assortativity_coefficient(multiplex)

-0.07063907775109278

### 4) Number of nodes in the Lagest Connected Component _Conn._

In [95]:
perc_nodes_in_lcc(multiplex)

1.0

### 5) Mean shortest path in the Largest Connected Component _d_

In [96]:
mean_shortest_path_lcc(multiplex)

2.391905825743255

In [97]:
pickle.dump(multiplex, open( "multiplex.pickle", "wb" ))

In [98]:
len(multiplex.nodes)

529

# Create the Word Embedding Network

In [99]:
glove_embedding = WordEmbeddings('glove')

In [100]:
word_emb = {}
for word in words:
    sentence = Sentence(word) # --> strip() removes the white space from beginning and end of word
     # embed a sentence using glove.
    glove_embedding.embed(sentence)
    for token in sentence:

        word_emb[word]=token.embedding

In [101]:
len(word_emb['red'])

100

In [102]:
pickle.dump(word_emb, open( "word_layer_vectors.pickle", "wb" ))

In [103]:
word_emb_layer = nx.Graph()
word_emb_layer.add_nodes_from(words)

### Compute distance using cosine similarity


In [104]:
word_emb_list = []
for item in tqdm(word_emb):
    x = word_emb[item]
   
    for word in word_emb:
        if word is not item:
            y = word_emb[word]
            cosine_sim = torch.cosine_similarity(x.unsqueeze(0), y.unsqueeze(0))
            
            if cosine_sim > word_thresh:
                word_emb_list.append((item, word))
                word_emb_layer.add_edge(item, word)

  0%|          | 0/529 [00:00<?, ?it/s]

In [105]:
len(word_emb_layer.nodes)

529

### 1) Mean Dregree of connectivity _k_

In [106]:
mean_degree_connectivity(word_emb_layer)

0.9300567107750473

### 2) Mean Clustering Coefficient _CC_

In [107]:
nx.average_clustering(word_emb_layer)

0.06606594537207236

### 3) Assortativity Coefficient _a_

In [108]:
nx.degree_assortativity_coefficient(word_emb_layer)

0.4112425700024269

### 4) Percentage of nodes in the LCC _Conn._

In [109]:
perc_nodes_in_lcc(word_emb_layer)

0.1342155009451796

### 5) Mean shortest path in LLC _d_

In [110]:
mean_shortest_path_lcc(word_emb_layer)

3.2305835010060364

In [111]:
pickle.dump(word_emb_list, open( "word_emb_layer.pickle", "wb" ))

In [112]:
len(word_emb_layer.nodes)

529

# Create the Visual Embedding Network

In [113]:
file = open('../data/clip_embeddings.txt', 'r')
lines = file.readlines()

In [114]:
visual_vecs = {}
for line in lines:
    visual_vecs[line.split()[0]] = line.split()[1:]

In [115]:
len(visual_vecs['car'])

513

In [116]:
pickle.dump(visual_vecs, open( "visual_layer_vectors.pickle", "wb" ))

In [117]:
visual_graph = nx.Graph()
visual_graph.add_nodes_from(words)
len(visual_graph.nodes)

529

In [118]:
visual_list = []
for item in tqdm(visual_vecs):
    x = np.array(visual_vecs[item])
   
    for word in visual_vecs:
        if word is not item:
            y = np.array(visual_vecs[word])
          
            cosine_sim = cosine_similarity(x.reshape(1,-1),y.reshape(1,-1))
            
            if cosine_sim > visual_thresh:
                visual_list.append((item, word))
                visual_graph.add_edge(item, word)

  0%|          | 0/496 [00:00<?, ?it/s]

In [119]:
len(visual_graph.nodes)

529

In [120]:
print(visual_list)

[('tree', 'plant'), ('tree', 'wolf'), ('tree', 'flower'), ('tree', 'tiger'), ('tree', 'airplane'), ('tree', 'flag'), ('tree', 'necklace'), ('horse', 'squirrel'), ('horse', 'wolf'), ('horse', 'tiger'), ('horse', 'airplane'), ('horse', 'moon'), ('horse', 'helicopter'), ('horse', 'flag'), ('horse', 'pony'), ('horse', 'penguin'), ('horse', 'sun'), ('horse', 'rooster'), ('horse', 'necklace'), ('horse', 'dog'), ('truck', 'wolf'), ('truck', 'airplane'), ('truck', 'tractor'), ('truck', 'helicopter'), ('truck', 'car'), ('plant', 'tree'), ('plant', 'wolf'), ('plant', 'flower'), ('plant', 'tiger'), ('plant', 'airplane'), ('plant', 'flag'), ('plant', 'garden'), ('plant', 'grass'), ('plant', 'butterfly'), ('plant', 'sun'), ('blue', 'eye'), ('blue', 'beads'), ('blue', 'wolf'), ('blue', 'dress'), ('blue', 'airplane'), ('blue', 'moon'), ('blue', 'helicopter'), ('blue', 'flag'), ('blue', 'button'), ('blue', 'penguin'), ('blue', 'light'), ('blue', 'yellow'), ('blue', 'necklace'), ('eye', 'blue'), ('eye'

### 1) Mean Degree of Connectivity _k_

In [121]:
mean_degree_connectivity(visual_graph)

1.1568998109640831

### 2) Mean Clustering Coefficient _CC_

In [122]:
nx.average_clustering(visual_graph)

0.06509661244878844

### 3) Assortativity Coefficient _a_

In [123]:
nx.degree_assortativity_coefficient(visual_graph)

-0.3743916999602581

### 4) Percentage of nodes in the LCC _Conn._

In [124]:
perc_nodes_in_lcc(visual_graph)

0.09829867674858223

### 5) Mean shortest length in the LCC _d_

In [125]:
mean_shortest_path_lcc(visual_graph)

1.9079939668174963

In [126]:
pickle.dump(visual_list, open( "visual_graph.pickle", "wb" ))

In [127]:
len(visual_graph.nodes)

529

# Create the Lancaster Embedding Network

In [128]:
norms = pd.read_csv('../data/Sensorimotor_norms.csv')
cols = norms.describe().columns

In [129]:
# normalize all columns
for col in cols:
    m = norms[col].max()
    norms[col] = norms[col] / m

In [130]:
vecs = {}

for i,row in norms.iterrows():
    vecs[row.Word.lower()] =  row[cols].values

In [131]:
len(vecs['red'])

22

In [132]:
pickle.dump(vecs, open( "lancaster_layer_vectors.pickle", "wb" ))

In [133]:
lancaster = nx.Graph()
lancaster.add_nodes_from(words)
len(lancaster.nodes)

529

In [134]:
lancaster_list = []

for item in tqdm(vecs):
    x =  vecs[item]

    
    for word in vecs:
        if word is not item:
            y =  vecs[word]
            cosine_sim = dot(x, y)/(norm(x)*norm(y))
            
            if cosine_sim > lancaster_thresh:
                
                if item not in words or word not in words:
                    continue
                else:
                    lancaster_list.append((item, word))
                    lancaster.add_edge(item, word)


  0%|          | 0/524 [00:00<?, ?it/s]

In [135]:
print(len(lancaster.nodes))

529


### 1) Mean Degree of Connectivity _k_

In [136]:
mean_degree_connectivity(lancaster)

6.930056710775047

### 2) Mean Clustering Coefficient _CC_

In [137]:
nx.average_clustering(lancaster)

0.25969123136243577

### 3) Assortativity Coefficient _a_

In [138]:
nx.degree_assortativity_coefficient(lancaster)

0.45154792499581853

### 4) Percetage of nodes in the LCC _Conn._

In [139]:
perc_nodes_in_lcc(lancaster)

0.5293005671077504

### 5) Mean shortest path length in LCC _d_

In [140]:
mean_shortest_path_lcc(lancaster)

3.615924219150026

In [141]:
pickle.dump(lancaster_list, open( "lancaster.pickle", "wb" ))

In [142]:
len(lancaster.nodes)

529

# Now, we will add the above created layers to the multiplex

## Word + Visual + Sensorimotor

In [143]:
new_3 = nx.Graph()

In [144]:
new_3.add_nodes_from(words)

In [145]:
for pair in visual_list:
    new_3.add_edge(pair[0], pair[1])
for pair in word_emb_list:
    new_3.add_edge(pair[0], pair[1])
for pair in lancaster_list:
    new_3.add_edge(pair[0], pair[1])

In [146]:
len(new_3.nodes)

529

### 1) k

In [147]:
mean_degree_connectivity(new_3)

8.81663516068053

### 2) CC

In [148]:
nx.average_clustering(new_3)

0.2954640058454584

### 3) a

In [149]:
nx.degree_assortativity_coefficient(new_3)

0.2821758755160438

### 4) Conn.

In [150]:
perc_nodes_in_lcc(new_3)

0.6275992438563327

### 5) d

In [151]:
mean_shortest_path_lcc(new_3)

3.4131328941142214

## Multiplex + new_3

In [152]:
multiplex_new_3 = multiplex.copy()

In [153]:
for pair in visual_list:
    multiplex_new_3.add_edge(pair[0], pair[1])
for pair in word_emb_list:
    multiplex_new_3.add_edge(pair[0], pair[1])
for pair in lancaster_list:
    multiplex_new_3.add_edge(pair[0], pair[1])

In [154]:
len(multiplex_new_3.nodes())

529

In [155]:
mean_degree_connectivity(multiplex_new_3)

33.342155009451794

In [156]:
nx.average_clustering(multiplex_new_3)

0.3330856221772822

In [157]:
nx.degree_assortativity_coefficient(multiplex_new_3)

-0.02013631469990666

In [158]:
perc_nodes_in_lcc(multiplex_new_3)

1.0

In [159]:
mean_shortest_path_lcc(multiplex_new_3)

2.2783052643638655

## Multiplex + Visual

### Make a copy of the multiplex to add each layer

In [160]:
multiplex_visual = multiplex.copy()

In [161]:
# multiplex + visual
for pair in visual_list:
    multiplex_visual.add_edge(pair[0], pair[1])

In [162]:
len(multiplex_visual.nodes)

529

### 1) _k_

In [163]:
mean_degree_connectivity(multiplex_visual)

27.330812854442343

### 2) _CC_


In [164]:
nx.average_clustering(multiplex_visual)

0.33048673859820415

### 3) _a_

In [165]:
nx.degree_assortativity_coefficient(multiplex_visual)

-0.06984398586988612

### 4) _Conn._

In [166]:
perc_nodes_in_lcc(multiplex_visual)

1.0

### 5)  _d_

In [167]:
mean_shortest_path_lcc(multiplex_visual)

2.375379503923927

## Multiplex + Word

In [168]:
multiplex_word = multiplex.copy()

In [169]:
for pair in word_emb_list:
    multiplex_word.add_edge(pair[0], pair[1])

In [170]:
len(multiplex_word.nodes)

529

### 1) _k_

In [171]:
mean_degree_connectivity(multiplex_word)

26.801512287334592

### 2) _CC_

In [172]:
nx.average_clustering(multiplex_word)

0.33161217183466807

### 3) _a_

In [173]:
nx.degree_assortativity_coefficient(multiplex_word)

-0.06763528402764445

### 4) _Conn._

In [174]:
perc_nodes_in_lcc(multiplex_word)

1.0

### 5) _d_

In [175]:
mean_shortest_path_lcc(multiplex_word)

2.389220656470184

# Multiplex + Lancaster

In [176]:
multiplex_lancaster = multiplex.copy()

In [177]:
for pair in lancaster_list:
    multiplex_lancaster.add_edge(pair[0], pair[1])

In [178]:
len(multiplex_lancaster.nodes)

529

### 1) _k_

In [179]:
mean_degree_connectivity(multiplex_lancaster)

32.189035916824196

### 2) _CC_

In [180]:
nx.average_clustering(multiplex_lancaster)

0.3278162045648223

### 3) _a_

In [181]:
nx.degree_assortativity_coefficient(multiplex_lancaster)

-0.021172720751662943

### 4) _Conn._

In [182]:
perc_nodes_in_lcc(multiplex_lancaster)

1.0

### 5) _d_

In [183]:
mean_shortest_path_lcc(multiplex_lancaster)

2.2954402245517556